In [11]:
import pandas as pd
import numpy as np
import time
import ccxt
from threading import Thread

In [12]:

exchange = ccxt.binance()
exchange

ccxt.binance()

In [13]:
class CCXTFuturesTrader(): 
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh,
                 units, position = 0, leverage = 5, sandbox = True):
        
        exchange.set_sandbox_mode(sandbox)
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.get_available_intervals()
        self.units = units
        self.position = position
        self.leverage = leverage
        self.cum_profits = 0
        
       
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
       
    
    def get_available_intervals(self):
        
        l = []
        for key, value in exchange.timeframes.items():
            l.append(key)
        self.available_intervals = l
    
    def start_trading(self, start = None, hist_bars = None):
        
        if not hist_bars:
            hist_bars = 1000
        
        exchange.set_leverage(leverage = self.leverage, symbol = self.symbol)
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 start = start, limit = hist_bars)
            thread = Thread(target = self.start_kline_stream, args = (self.stream_candles, self.symbol, self.bar_length))
            thread.start()
            
       
    
    def get_most_recent(self, symbol, interval, start, limit):
        
        if start:
            start = exchange.parse8601(start)
    
        data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
        last_bar_actual = data[-1][0]
    
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
        while last_bar_target != last_bar_actual:
        
            time.sleep(0.1)
            data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = last_bar_actual, limit = limit)
            data += data_add[1:]
            last_bar_actual = data[-1][0]
            last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
        df = pd.DataFrame(data)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        df.Date = pd.to_datetime(df.Date, unit = "ms")
        df.set_index("Date", inplace = True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar = df.index[-1]

        self.data = df
        
    def stream_candles(self, msg):
        
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        if start_time < self.last_bar:
            pass
            
        else:
            if start_time == self.last_bar:
                complete = False
            elif start_time > self.last_bar:
                complete = True
                if len(msg) == 2:
                    self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
                else:
                    self.data.loc[self.last_bar, "Complete"] = complete
                self.last_bar = start_time
            
           
            print(".", end = "", flush = True)
        
        
            self.data.loc[start_time] = [first, high, low, close, volume, False]
            if complete == True:
                self.define_strategy()
                self.execute_trades()
    
    def start_kline_stream(self, callback, symbol, interval):
    
        self.running = True
    
        while self.running == True:
            msg = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)
        
            if len(msg) == 0:
                print("No data received")
            else:
                callback(msg)
    
            time.sleep(1)
    
    def stop_stream(self):
        self.running = False     
         
    def define_strategy(self):
        
        df = self.data.loc[self.data.Complete == True].copy()
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns <= self.return_thresh[0]
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        cond3 = df.returns >= self.return_thresh[1]
        
        df["position"] = 0
        df.loc[cond1 & cond2, "position"] = 1
        df.loc[cond3 & cond2, "position"] = -1
       
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = 2 * self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "BUY", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "SELL", amount = 2 * self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
    
    def report_trade(self, order, going): 
        
        time.sleep(0.1)
        order_time = int(order["info"]["updateTime"])
        trades = exchange.fetchMyTrades(symbol = self.symbol, since = order_time)
        order_time = pd.to_datetime(order_time, unit = "ms")
        
       
        df = pd.json_normalize(trades)
        columns = ["amount", "cost", "info.commission","info.realizedPnl"]
        for column in columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        base_units = round(df["amount"].sum(), 5)
        quote_units = round(df["cost"].sum(), 5)
        commission = -round(df["info.commission"].sum(), 5)
        real_profit = round(df["info.realizedPnl"].sum(), 5)
        price = round(quote_units / base_units, 5)
        
    
        self.cum_profits += round((commission + real_profit), 5)
        
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(order_time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(order_time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(order_time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [14]:
api_key = "f292676b40a6678d07e5693490e27730269394a447def962459564aa187f003a"
secret_key = "5b0d1ab790adec74a936873b2a3327e11b4987c9b6ad94e0fbe397f93ee0120f"#put your api key and secret key of testnet

In [15]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [16]:
symbol = "BTC/USDT"
bar_length = "1m"
return_thresh = [-0.0001, 0.0001]
volume_thresh = [-3, 3]
units = 0.01
position = 0
leverage = 5
sandbox = True

In [17]:
trader = CCXTFuturesTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                           volume_thresh = volume_thresh, units = units, position = position,
                           leverage = leverage, sandbox = sandbox)

In [18]:
exchange.fetchBalance()["info"]["assets"]

AuthenticationError: binance {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action."}

In [ ]:
trader.start_trading(start = None, hist_bars = 10)

In [ ]:
trader.stop_stream()

In [ ]:
trader.prepared_data

In [ ]:
exchange.fetchBalance()["info"]["assets"] # get asset balances